# Memory Agent

## Review

We created a chatbot that saves semantic memories to a single [user profile](https://langchain-ai.github.io/langgraph/concepts/memory/#profile) or [collection](https://langchain-ai.github.io/langgraph/concepts/memory/#collection).

We introduced [Trustcall](https://github.com/hinthornw/trustcall) as a way to update either schema.

## Goals

Now, we're going to pull together the pieces we've learned to build an [agent](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/) with long-term memory.

Our agent, `task_mAIstro`, will help us manage a ToDo list! 

The chatbots we built previously *always* reflected on the conversation and saved memories. 

`task_mAIstro` will decide *when* to save memories (items to our ToDo list).

The chatbots we built previously always saved one type of memory, a profile or a collection. 

`task_mAIstro` can decide to save to either a user profile or a collection of ToDo items.

In addition semantic memory, `task_mAIstro` also will manage procedural memory.

This allows the user to update their preferences for creating ToDo items. 

In [1]:
import os, getpass

def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")
    
    # Set the environment variable for the current process
    os.environ[var] = env_value

_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

In [2]:
_set_env("OPENAI_API_KEY")

## Visibility into Trustcall updates

Trustcall creates and updates JSON schemas.

What if we want visibility into the *specific changes* made by Trustcall?

For example, we saw before that Trustcall has some of its own tools to:

* Self-correct from validation failures -- [see trace example here](https://smith.langchain.com/public/5cd23009-3e05-4b00-99f0-c66ee3edd06e/r/9684db76-2003-443b-9aa2-9a9dbc5498b7) 
* Update existing documents -- [see trace example here](https://smith.langchain.com/public/f45bdaf0-6963-4c19-8ec9-f4b7fe0f68ad/r/760f90e1-a5dc-48f1-8c34-79d6a3414ac3)

Visibility into these tools can be useful for the agent we're going to build.

Below, we'll show how to do this!

In [3]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    content: str = Field(description="The main content of the memory. For example: User expressed interest in learning about French.")

class MemoryCollection(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about the user.")

We can add a [listener](https://python.langchain.com/docs/how_to/lcel_cheatsheet/#add-lifecycle-listeners) to the Trustcall extractor.

This will pass runs from the extractor's execution to a class, `Spy`, that we will define.

Our `Spy` class will extract information about what tool calls were made by Trustcall.

In [4]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI

# Inspect the tool calls made by Trustcall
class Spy:
    def __init__(self):
        self.called_tools = []

    def __call__(self, run):
        # Collect information about the tool calls made by the extractor.
        q = [run]
        while q:
            r = q.pop()
            if r.child_runs:
                q.extend(r.child_runs)
            if r.run_type == "chat_model":
                self.called_tools.append(
                    r.outputs["generations"][0][0]["message"]["kwargs"]["tool_calls"]
                )

# Initialize the spy
spy = Spy()

# Initialize the model
model = ChatOpenAI(model="gpt-4o", temperature=0)

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,
)

# Add the spy as a listener
trustcall_extractor_see_all_tool_calls = trustcall_extractor.with_listeners(on_end=spy)

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

instruction = """Extract memories from the following conversation: """

conversation = [HumanMessage(content="Hi, I'm Charlie"),
                AIMessage(content="Nice to meet you, Charlie"),
                HumanMessage(content="This morning I had a nice walk in Portland, OR")               
               ]

result = trustcall_extractor.invoke({"messages": [SystemMessage(content=instruction)] + conversation})

In [6]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_lJNl4rthw2STuH1GEWidDuZo)
 Call ID: call_lJNl4rthw2STuH1GEWidDuZo
  Args:
    content: Charlie had a nice walk in Portland, OR this morning.


In [8]:
for m in result["responses"]:
    print(m)

content='Charlie had a nice walk in Portland, OR this morning.'


In [9]:
for m in result["response_metadata"]:
    print(m)

{'id': 'call_lJNl4rthw2STuH1GEWidDuZo'}


In [10]:
updated_conversation = [AIMessage(content="That's great, did you do after?"), 
                        HumanMessage(content="I went to my piano lesson."),                        
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my upcoming trip to Oceanside, OR."),]

system_msg = """Update existing memories and create new ones based on the following conversation:"""

tool_name = "Memory"
existing_memories = [
    (str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])
] if result["responses"] else None
existing_memories

[('0',
  'Memory',
  {'content': 'Charlie had a nice walk in Portland, OR this morning.'})]

In [11]:
result = trustcall_extractor_see_all_tool_calls.invoke({
    "messages": updated_conversation,
    "existing": existing_memories,
})

In [12]:
for m in result["response_metadata"]:
    print(m)

{'id': 'call_P3GiuIgGILW8Kyg2EZYmn9P8', 'json_doc_id': '0'}
{'id': 'call_VEsDMaTLy7NkMhrK1BzcDVSV'}
{'id': 'call_UTTxeSzlkIvdjS8MYDStkh1B'}


In [13]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_P3GiuIgGILW8Kyg2EZYmn9P8)
 Call ID: call_P3GiuIgGILW8Kyg2EZYmn9P8
  Args:
    content: Charlie had a nice walk in Portland, OR this morning. Then, Charlie went to a piano lesson. Charlie is also thinking about an upcoming trip to Oceanside, OR.
  Memory (call_VEsDMaTLy7NkMhrK1BzcDVSV)
 Call ID: call_VEsDMaTLy7NkMhrK1BzcDVSV
  Args:
    content: Charlie went to a piano lesson.
  Memory (call_UTTxeSzlkIvdjS8MYDStkh1B)
 Call ID: call_UTTxeSzlkIvdjS8MYDStkh1B
  Args:
    content: Charlie is thinking about an upcoming trip to Oceanside, OR.


In [14]:
for m in result["responses"]:
    print(m)

content='Charlie had a nice walk in Portland, OR this morning. Then, Charlie went to a piano lesson. Charlie is also thinking about an upcoming trip to Oceanside, OR.'
content='Charlie went to a piano lesson.'
content='Charlie is thinking about an upcoming trip to Oceanside, OR.'


In [21]:
for i in spy.called_tools[0]:
    print(i, "\n")


{'name': 'PatchDoc', 'args': {'json_doc_id': '0', 'planned_edits': "Update the existing Memory instance to include the new information about the piano lesson and the upcoming trip to Oceanside, OR. The 'content' field in the Memory schema should be updated to reflect these new details.", 'patches': [{'op': 'replace', 'path': '/content', 'value': 'Charlie had a nice walk in Portland, OR this morning. Then, Charlie went to a piano lesson. Charlie is also thinking about an upcoming trip to Oceanside, OR.'}]}, 'id': 'call_P3GiuIgGILW8Kyg2EZYmn9P8', 'type': 'tool_call'} 

{'name': 'Memory', 'args': {'content': 'Charlie went to a piano lesson.'}, 'id': 'call_VEsDMaTLy7NkMhrK1BzcDVSV', 'type': 'tool_call'} 

{'name': 'Memory', 'args': {'content': 'Charlie is thinking about an upcoming trip to Oceanside, OR.'}, 'id': 'call_UTTxeSzlkIvdjS8MYDStkh1B', 'type': 'tool_call'} 



In [22]:
def extract_tool_info(tool_calls, schema_name="Memory"):
    """Extract information from tool calls for both patches and new memories.
    
    Args:
        tool_calls: List of tool calls from the model
        schema_name: Name of the schema tool (e.g., "Memory", "ToDo", "Profile")
    """

    # Initialize list of changes
    changes = []
    
    for call_group in tool_calls:
        for call in call_group:
            if call['name'] == 'PatchDoc':
                changes.append({
                    'type': 'update',
                    'doc_id': call['args']['json_doc_id'],
                    'planned_edits': call['args']['planned_edits'],
                    'value': call['args']['patches'][0]['value']
                })
            elif call['name'] == schema_name:
                changes.append({
                    'type': 'new',
                    'value': call['args']
                })

    # Format results as a single string
    result_parts = []
    for change in changes:
        if change['type'] == 'update':
            result_parts.append(
                f"Document {change['doc_id']} updated:\n"
                f"Plan: {change['planned_edits']}\n"
                f"Added content: {change['value']}"
            )
        else:
            result_parts.append(
                f"New {schema_name} created:\n"
                f"Content: {change['value']}"
            )
    
    return "\n\n".join(result_parts)

# Inspect spy.called_tools to see exactly what happened during the extraction
schema_name = "Memory"
changes = extract_tool_info(spy.called_tools, schema_name)
print(changes)

Document 0 updated:
Plan: Update the existing Memory instance to include the new information about the piano lesson and the upcoming trip to Oceanside, OR. The 'content' field in the Memory schema should be updated to reflect these new details.
Added content: Charlie had a nice walk in Portland, OR this morning. Then, Charlie went to a piano lesson. Charlie is also thinking about an upcoming trip to Oceanside, OR.

New Memory created:
Content: {'content': 'Charlie went to a piano lesson.'}

New Memory created:
Content: {'content': 'Charlie is thinking about an upcoming trip to Oceanside, OR.'}


## Creating an agent

There are many different [agent](https://langchain-ai.github.io/langgraph/concepts/high_level/) architectures to choose from.

Here, we'll implement something simple, a [ReAct](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#react-implementation) agent.

This agent will be a helpful companion for creating and managing a ToDo list.

This agent can make a decision to update three types of long-term memory: 

(a) Create or update a user `profile` with general user information 

(b) Add or update items in a ToDo list `collection`

(c) Update its own `instructions` on how to update items to the ToDo list

## Graph definition 

We add a simple router, `route_message`, that makes a binary decision to save memories.

The memory collection updating is handled by `Trustcall` in the `write_memory` node, as before!

We can see that Trustcall performs patching of the existing memory:

https://smith.langchain.com/public/4ad3a8af-3b1e-493d-b163-3111aa3d575a/r

Now we can create a new thread.

This creates a new session. 

Profile, ToDos, and Instructions saved to long-term memory are accessed. 

Trace: 

https://smith.langchain.com/public/84768705-be91-43e4-8a6f-f9d3cee93782/r

## Studio

![Screenshot 2024-11-04 at 1.00.19 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/6732cfb05d9709862eba4e6c_Screenshot%202024-11-11%20at%207.46.40%E2%80%AFPM.png)